In [2]:
import re  # 정규표현식 사용                                     
from collections import Counter  # 단어 빈도 계산용                      

TEXT = "Hello! This is a test 문장입니다. NLP 전처리: 정규화/토큰화/인코딩 😊"  # 예시 텍스트      
print(TEXT)
def normalize(text: str) -> str:  # 정규화 함수(소문자화/특수문자 제거)      
    text = text.lower()  # 소문자화                                       
    text = re.sub(r"[^\w\s\.!?가-힣]", " ", text)  # 특수문자 제거(일부 문장부호는 유지) 
    text = re.sub(r"\s+", " ", text).strip()  # 공백 정리                    
    return text  # 정규화된 텍스트 반환                                    

def split_sentences(text: str) -> list[str]:  # 문장 분리 함수               
    parts = re.split(r"(?<=[\.\!\?])\s+", text)  # .!? 뒤에서 분리           
    parts = [s.strip() for s in parts if s.strip()]  # 빈 문장 제거          
    return parts  # 문장 리스트 반환                                        

def tokenize_word(sentence: str) -> list[str]:  # 워드 토큰화                
    return re.findall(r"[A-Za-z0-9_]+|[가-힣]+", sentence)  # 영문/숫자/한글 단어 추출

def tokenize_char(sentence: str) -> list[str]:  # 캐릭터 토큰화              
    return list(sentence)  # 글자 단위로 쪼갬                               

def tokenize_subword_simple(words: list[str]) -> list[str]:  # “간단” 서브워드 토큰화(데모) 
    pieces = []  # 결과 조각 리스트                                         
    for w in words:  # 각 단어에 대해                                      
        if len(w) <= 4:  # 짧으면 그대로 사용                                
            pieces.append(w)  # 그대로 추가                                 
        else:  # 길면 앞/뒤로 대충 분해(진짜 BPE/WordPiece 대체용 데모)       
            pieces.append(w[:3])  # 앞 3글자                                 
            pieces.append("##" + w[3:])  # 나머지(접두 표기 흉내)             
    return pieces  # 서브워드 조각 반환                                     

STOPWORDS = {  # 불용어(데모용, 필요에 맞게 확장)                            
    "is", "a", "the", "this", "to", "of", "and", "in", "on", "for",  # 영어 불용어 
    "은", "는", "이", "가", "을", "를", "에", "의", "와", "과", "도"  # 한글 조사 일부 
}  # 불용어 집합 끝                                                        

def remove_stopwords(tokens: list[str]) -> list[str]:  # 불용어 제거 함수     
    return [t for t in tokens if t not in STOPWORDS]  # 불용어 제외          

def stem_simple(token: str) -> str:  # 아주 단순한 어간 추출(영어 위주 데모)  
    t = token  # 작업용 복사                                                
    for suf in ("ing", "ed", "es", "s"):  # 흔한 접미사 목록                  
        if len(t) > 4 and t.endswith(suf):  # 너무 짧은 단어는 보호            
            t = t[: -len(suf)]  # 접미사 제거                                
            break  # 하나만 제거하고 종료                                   
    return t  # 스템 결과 반환                                              

LEMMA_MAP = {  # 표제어 추출(간단 매핑 데모: 실제론 사전/모델 필요)           
    "mice": "mouse",  # 예시 매핑                                          
    "children": "child",  # 예시 매핑                                      
    "went": "go",  # 예시 매핑                                              
}  # 매핑 끝                                                                

def lemmatize_simple(token: str) -> str:  # 아주 단순한 표제어 추출(데모)     
    return LEMMA_MAP.get(token, token)  # 매핑 있으면 치환, 없으면 그대로     

def build_vocab(token_sequences: list[list[str]], min_freq: int = 1) -> dict[str, int]:  # 어휘집합 생성
    counts = Counter(t for seq in token_sequences for t in seq)  # 전체 토큰 빈도 집계 
    vocab = {"[PAD]": 0, "[UNK]": 1}  # 특수 토큰(패딩/미등록) 기본 등록         
    for tok, c in counts.most_common():  # 빈도 높은 순으로 순회               
        if c >= min_freq and tok not in vocab:  # 최소 빈도 이상만              
            vocab[tok] = len(vocab)  # 새 ID 부여                            
    return vocab  # vocab(토큰→id) 반환                                     

def integer_encode(tokens: list[str], vocab: dict[str, int]) -> list[int]:  # 정수 인코딩
    return [vocab.get(t, vocab["[UNK]"]) for t in tokens]  # 모르면 UNK       

def pad_and_mask(ids: list[int], max_len: int, pad_id: int = 0) -> tuple[list[int], list[int]]:  # 패딩/마스킹
    ids = ids[:max_len]  # 길면 자르기                                      
    mask = [1] * len(ids)  # 실제 토큰은 1                                   
    while len(ids) < max_len:  # 짧으면 패딩 추가                             
        ids.append(pad_id)  # PAD 추가                                       
        mask.append(0)  # PAD 위치는 0                                       
    return ids, mask  # (패딩된 ids, 마스크) 반환                            

# --------- “통으로” 파이프라인 실행 예시 ---------  # 구분선                 
norm = normalize(TEXT)  # 1) 정규화                                         
sents = split_sentences(norm)  # 2) 문장 분리                               

word_tokens = [tokenize_word(s) for s in sents]  # 3) 워드 토큰화(문장별)      
word_tokens = [remove_stopwords(ws) for ws in word_tokens]  # 4) 불용어 제거(문장별)

stemmed = [[stem_simple(t) for t in ws] for ws in word_tokens]  # 5) 어간 추출(문장별)
lemmatized = [[lemmatize_simple(t) for t in ws] for ws in word_tokens]  # 6) 표제어(문장별)

subword_tokens = [tokenize_subword_simple(ws) for ws in word_tokens]  # 7) 서브워드 토큰화(데모)
char_tokens = [tokenize_char(s) for s in sents]  # 8) 캐릭터 토큰화(문장별)    

vocab = build_vocab(subword_tokens, min_freq=1)  # 9) 어휘집합(vocab) 생성(여기선 서브워드 기준)
encoded = [integer_encode(seq, vocab) for seq in subword_tokens]  # 10) 정수 인코딩(문장별)

max_len = 12  # 11) 패딩 기준 길이                                          
padded_and_masked = [pad_and_mask(seq, max_len, pad_id=vocab["[PAD]"]) for seq in encoded]  # 12) 패딩/마스킹

print("정규화:", norm)  # 결과 출력                                         
print("문장:", sents)  # 결과 출력                                          
print("워드:", word_tokens)  # 결과 출력                                    
print("어간:", stemmed)  # 결과 출력                                        
print("표제어:", lemmatized)  # 결과 출력                                   
print("서브워드(데모):", subword_tokens)  # 결과 출력                        
print("vocab size:", len(vocab))  # 결과 출력                                
print("vocab:", vocab)  # 결과 출력                                
print("인코딩:", encoded)  # 결과 출력                                      
print("패딩/마스크:", padded_and_masked)  # 결과 출력                        


Hello! This is a test 문장입니다. NLP 전처리: 정규화/토큰화/인코딩 😊
정규화: hello! this is a test 문장입니다. nlp 전처리 정규화 토큰화 인코딩
문장: ['hello!', 'this is a test 문장입니다.', 'nlp 전처리 정규화 토큰화 인코딩']
워드: [['hello'], ['test', '문장입니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
어간: [['hello'], ['test', '문장입니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
표제어: [['hello'], ['test', '문장입니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
서브워드(데모): [['hel', '##lo'], ['test', '문장입', '##니다'], ['nlp', '전처리', '정규화', '토큰화', '인코딩']]
vocab size: 12
vocab: {'[PAD]': 0, '[UNK]': 1, 'hel': 2, '##lo': 3, 'test': 4, '문장입': 5, '##니다': 6, 'nlp': 7, '전처리': 8, '정규화': 9, '토큰화': 10, '인코딩': 11}
인코딩: [[2, 3], [4, 5, 6], [7, 8, 9, 10, 11]]
패딩/마스크: [([2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ([4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ([7, 8, 9, 10, 11, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])]


In [3]:
import re  # 정규표현식                                                   
import math  # 수학 함수                                                 
import numpy as np  # 벡터/행렬 계산                                     
from collections import Counter, defaultdict  # 빈도/맵핑                 

# =========================
# 0) 예시 텍스트
# =========================
TEXTS = [  # 간단 코퍼스                                                 
    "먹었습니다",  # 형태소 예시                                         
    "먹는 중입니다",  # 형태소 예시                                     
    "자연어처리는 재미있습니다",  # 형태소 예시                          
    "I love tokenization and NLP",  # 서브워드/임베딩 예시               
    "tokenization makes models robust",  # 서브워드 예시                 
]  # 코퍼스 끝                                                           

# =========================
# 1) 형태소 분석(데모): 조사/어미 분리 + POS 흉내
#    ※ 실제론 Mecab/Komoran/OKT 등 형태소 분석기 사용
# =========================
KOREAN_ENDINGS = [  # 아주 일부 어미/종결(데모)                           
    ("었습니다", "EF"),  # 종결 어미 흉내                                
    ("습니다", "EF"),  # 종결 어미 흉내                                
    ("입니다", "EF"),  # 종결 어미 흉내                                
    ("는", "ETM"),  # 관형사형 어미 흉내                                
    ("었", "EP"),  # 선어말 어미 흉내                                   
]  # 어미 목록 끝                                                        

KOREAN_PARTICLES = [  # 아주 일부 조사(데모)                              
    ("을", "JKO"), ("를", "JKO"),  # 목적격 조사                          
    ("은", "JX"), ("는", "JX"),  # 보조사                                
    ("이", "JKS"), ("가", "JKS"),  # 주격 조사                            
    ("에", "JKB"), ("의", "JKG"),  # 부사격/관형격 조사                   
]  # 조사 목록 끝                                                         

def morph_analyze_demo(token: str) -> list[tuple[str, str]]:  # 형태소 분석(데모)  
    t = token.strip()  # 공백 제거                                       
    out = []  # 결과                                                     
    for suf, pos in sorted(KOREAN_PARTICLES, key=lambda x: -len(x[0])):  # 긴 조사부터
        if t.endswith(suf) and len(t) > len(suf):  # 조사로 끝나면        
            stem = t[:-len(suf)]  # 앞부분                               
            out.append((stem, "NNG/VV?"))  # 어간(명사/동사?) 데모         
            out.append((suf, pos))  # 조사                               
            return out  # 종료                                            
    for suf, pos in sorted(KOREAN_ENDINGS, key=lambda x: -len(x[0])):  # 긴 어미부터
        if t.endswith(suf) and len(t) > len(suf):  # 어미로 끝나면        
            stem = t[:-len(suf)]  # 어간                                 
            out.append((stem, "VV/VA?"))  # 어간(동사/형용사?) 데모        
            out.append((suf, pos))  # 어미                               
            return out  # 종료                                            
    return [(t, "NNG/VV?")]  # 못 자르면 통째로                          

print("=== 1) 형태소 분석(데모) ===")  # 섹션 출력                         
for s in ["먹었습니다", "먹는", "학생을", "자연어처리는"]:  # 예시            
    print(s, "->", " + ".join([f"{m}/{p}" for m, p in morph_analyze_demo(s)]))  # 출력

# =========================
# 2) 진짜 서브워드: BPE(학습) + WordPiece 스타일(##) 토큰화
# =========================

def basic_normalize_en(text: str) -> list[str]:  # 영어 토큰화(데모)      
    return re.findall(r"[a-zA-Z]+", text.lower())  # 단어만 추출           

CORPUS_WORDS = []  # BPE 학습용 단어 리스트                               
for t in TEXTS:  # 코퍼스 순회                                            
    CORPUS_WORDS += basic_normalize_en(t)  # 영어 단어 누적                

def build_bpe_vocab(words: list[str]) -> dict[tuple[str, ...], int]:  # BPE용 vocab(단어->빈도)
    counts = Counter(words)  # 단어 빈도                                  
    vocab = {}  # {(문자 시퀀스): 빈도}                                  
    for w, c in counts.items():  # 각 단어                                
        seq = tuple(list(w) + ["</w>"])  # 문자 단위 + 종료 표시           
        vocab[seq] = c  # 빈도 저장                                       
    return vocab  # 반환                                                  

def get_pair_stats(vocab: dict[tuple[str, ...], int]) -> Counter:  # bigram 통계
    stats = Counter()  # (a,b) 빈도                                       
    for seq, c in vocab.items():  # 각 단어 시퀀스                         
        for i in range(len(seq) - 1):  # 인접쌍                            
            stats[(seq[i], seq[i+1])] += c  # 가중 빈도                    
    return stats  # 반환                                                  

def merge_pair(vocab: dict[tuple[str, ...], int], pair: tuple[str, str]) -> dict[tuple[str, ...], int]:  # 병합
    a, b = pair  # 병합 대상                                               
    new_vocab = {}  # 새 vocab                                             
    ab = a + b  # 합친 토큰                                                 
    for seq, c in vocab.items():  # 각 시퀀스                              
        new_seq = []  # 병합 결과                                          
        i = 0  # 인덱스                                                    
        while i < len(seq):  # 순회                                        
            if i < len(seq) - 1 and seq[i] == a and seq[i+1] == b:  # 쌍이면
                new_seq.append(ab)  # 병합 토큰 추가                       
                i += 2  # 2칸 이동                                        
            else:  # 아니면                                                
                new_seq.append(seq[i])  # 그대로 추가                      
                i += 1  # 1칸 이동                                        
        new_vocab[tuple(new_seq)] = new_vocab.get(tuple(new_seq), 0) + c  # 누적
    return new_vocab  # 반환                                               

def train_bpe(words: list[str], num_merges: int = 20) -> tuple[list[tuple[str, str]], set[str]]:  # BPE 학습
    vocab = build_bpe_vocab(words)  # 초기 문자 vocab                      
    merges = []  # 병합 기록                                               
    for _ in range(num_merges):  # 병합 반복                               
        stats = get_pair_stats(vocab)  # 쌍 빈도                           
        if not stats:  # 더 이상 없으면                                    
            break  # 종료                                                  
        best = stats.most_common(1)[0][0]  # 최빈 쌍                        
        merges.append(best)  # 기록                                        
        vocab = merge_pair(vocab, best)  # 병합 적용                        
    token_set = set()  # 최종 토큰 집합                                    
    for seq in vocab.keys():  # vocab의 시퀀스들                            
        for tok in seq:  # 각 토큰                                         
            token_set.add(tok)  # 추가                                     
    return merges, token_set  # 반환                                       

def bpe_encode(word: str, merges: list[tuple[str, str]]) -> list[str]:  # BPE 인코딩(단어 단위)
    seq = list(word) + ["</w>"]  # 문자 + 종료                             
    for a, b in merges:  # 병합 순서대로                                   
        i = 0  # 인덱스                                                    
        new_seq = []  # 병합 결과                                          
        while i < len(seq):  # 순회                                        
            if i < len(seq) - 1 and seq[i] == a and seq[i+1] == b:  # 쌍이면
                new_seq.append(a + b)  # 합친 토큰                          
                i += 2  # 이동                                            
            else:  # 아니면                                                
                new_seq.append(seq[i])  # 그대로                           
                i += 1  # 이동                                            
        seq = new_seq  # 갱신                                              
    if seq and seq[-1] == "</w>":  # 종료 토큰 제거                         
        seq = seq[:-1]  # 제거                                            
    return seq  # 반환                                                     

# WordPiece 스타일 토큰화(##): "단어 내부 조각"을 ##로 표시
def wordpiece_style(pieces: list[str]) -> list[str]:  # ## 스타일 변환     
    if not pieces:  # 빈 입력                                              
        return []  # 반환                                                  
    out = [pieces[0]]  # 첫 조각은 그대로                                  
    for p in pieces[1:]:  # 나머지 조각                                    
        out.append("##" + p)  # 뒤 조각은 ## 표시                          
    return out  # 반환                                                     

print("\n=== 2) BPE 학습 + WordPiece(##) 스타일 ===")  # 섹션 출력           
MERGES, TOKSET = train_bpe(CORPUS_WORDS, num_merges=30)  # BPE 학습        
print("학습 단어들:", CORPUS_WORDS)  # 코퍼스 단어                         
print("병합 일부:", MERGES[:10])  # 병합 앞부분                            
for w in ["tokenization", "models", "robust"]:  # 테스트 단어              
    bpe = bpe_encode(w, MERGES)  # BPE 조각                                
    wp = wordpiece_style(bpe)  # ## 스타일                                 
    print(w, "-> BPE:", bpe, "-> WordPiece-style:", wp)  # 출력            

# =========================
# 3) 임베딩: token id -> vector (공간)
# =========================
def build_vocab(tokens: list[str]) -> dict[str, int]:  # vocab 생성        
    vocab = {"[PAD]": 0, "[UNK]": 1}  # 특수 토큰                          
    for t in tokens:  # 토큰 순회                                          
        if t not in vocab:  # 없으면                                       
            vocab[t] = len(vocab)  # id 부여                               
    return vocab  # 반환                                                   

def encode(tokens: list[str], vocab: dict[str, int]) -> list[int]:  # id 인코딩
    return [vocab.get(t, vocab["[UNK]"]) for t in tokens]  # UNK 처리       

def cosine(a: np.ndarray, b: np.ndarray) -> float:  # 코사인 유사도        
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-9))  # 안정화

print("\n=== 3) 임베딩(공간) ===")  # 섹션 출력                              
sample_tokens = wordpiece_style(bpe_encode("tokenization", MERGES))  # 예시 토큰
vocab = build_vocab(sample_tokens)  # vocab 생성                           
ids = encode(sample_tokens, vocab)  # id 변환                              
d = 8  # 임베딩 차원(데모)                                                 
E = np.random.randn(len(vocab), d)  # 임베딩 행렬(토큰 수 x 차원)           
vecs = E[ids]  # (길이 x d)                                                
print("tokens:", sample_tokens)  # 토큰 출력                               
print("ids   :", ids)  # id 출력                                          
print("vecs shape:", vecs.shape)  # 임베딩 텐서 크기                       

if len(vecs) >= 2:  # 비교 가능하면                                       
    print("cosine(token[0], token[1]) =", cosine(vecs[0], vecs[1]))  # 공간 유사도

# =========================
# 4) 포지션 임베딩 + attention mask 의미(패딩 영향 차단)
#    - toy self-attention(1-head) 구현
# =========================
def softmax(x: np.ndarray, axis: int = -1) -> np.ndarray:  # 소프트맥스    
    x = x - np.max(x, axis=axis, keepdims=True)  # 안정화                  
    e = np.exp(x)  # 지수                                                  
    return e / (np.sum(e, axis=axis, keepdims=True) + 1e-9)  # 정규화       

def add_positional_embedding(X: np.ndarray) -> np.ndarray:  # 포지션 임베딩(사인/코사인)
    L, D = X.shape  # 길이/차원                                            
    P = np.zeros((L, D))  # 포지션 행렬                                    
    for pos in range(L):  # 위치                                           
        for i in range(0, D, 2):  # 짝/홀                                   
            denom = 10000 ** (i / D)  # 분모                               
            P[pos, i] = math.sin(pos / denom)  # sin                        
            if i + 1 < D:  # 범위 체크                                     
                P[pos, i+1] = math.cos(pos / denom)  # cos                 
    return X + P  # 더해서 반환                                            

def self_attention_toy(X: np.ndarray, mask: np.ndarray) -> tuple[np.ndarray, np.ndarray]:  # 어텐션(데모)
    D = X.shape[1]  # 차원                                                 
    Wq = np.random.randn(D, D)  # Q 가중치                                 
    Wk = np.random.randn(D, D)  # K 가중치                                 
    Wv = np.random.randn(D, D)  # V 가중치                                 
    Q = X @ Wq  # (L,D)                                                    
    K = X @ Wk  # (L,D)                                                    
    V = X @ Wv  # (L,D)                                                    
    scores = (Q @ K.T) / math.sqrt(D)  # (L,L) 점수                         
    # mask: 1=유효, 0=PAD -> PAD 위치로 가는 attention을 매우 작은 값으로 막음
    pad_to = (1 - mask)[None, :]  # (1,L) PAD 위치                          
    scores = scores + pad_to * (-1e9)  # PAD로 가는 경로 차단               
    A = softmax(scores, axis=-1)  # attention weights                       
    Y = A @ V  # 컨텍스트 결과 (L,D)                                       
    return Y, A  # 출력/가중치 반환                                         

print("\n=== 4) 포지션 + 마스크(Self-Attention) ===")  # 섹션 출력           
# 예시: 길이 6으로 패딩(뒤 2개 PAD)                                        
max_len = 6  # 최대 길이                                                   
pad_id = vocab["[PAD]"]  # PAD id                                          
ids_padded = ids[:max_len]  # 자르기                                       
mask = np.ones(len(ids_padded), dtype=np.float32)  # 마스크(유효=1)         
while len(ids_padded) < max_len:  # 패딩 추가                              
    ids_padded.append(pad_id)  # PAD 추가                                  
    mask = np.append(mask, 0.0)  # PAD 마스크=0                             

X = E[ids_padded]  # 임베딩 조회 (L,D)                                     
X_pos = add_positional_embedding(X)  # 포지션 추가                          
Y, A = self_attention_toy(X_pos, mask)  # 마스크 적용 어텐션                

print("ids_padded:", ids_padded)  # 패딩 id                                
print("mask      :", mask.tolist())  # 마스크                              
print("attention row0(sum):", float(np.sum(A[0])))  # 확률합=1 확인          
print("attention row0 to PAD idx:", [float(A[0, i]) for i in range(max_len) if mask[i] == 0.0])  # PAD로 거의 0

# (비교) 마스크 없이 돌리면 PAD에도 attention이 새어 나갈 수 있음           
Y2, A2 = self_attention_toy(X_pos, np.ones_like(mask))  # 마스크 없는 척    
print("no-mask row0 to PAD idx:", [float(A2[0, i]) for i in range(max_len) if mask[i] == 0.0])  # PAD로 새는 값


=== 1) 형태소 분석(데모) ===
먹었습니다 -> 먹/VV/VA? + 었습니다/EF
먹는 -> 먹/NNG/VV? + 는/JX
학생을 -> 학생/NNG/VV? + 을/JKO
자연어처리는 -> 자연어처리/NNG/VV? + 는/JX

=== 2) BPE 학습 + WordPiece(##) 스타일 ===
학습 단어들: ['i', 'love', 'tokenization', 'and', 'nlp', 'tokenization', 'makes', 'models', 'robust']
병합 일부: [('k', 'e'), ('t', 'o'), ('to', 'ke'), ('toke', 'n'), ('token', 'i'), ('tokeni', 'z'), ('tokeniz', 'a'), ('tokeniza', 't'), ('tokenizat', 'i'), ('tokenizati', 'o')]
tokenization -> BPE: ['tokenization</w>'] -> WordPiece-style: ['tokenization</w>']
models -> BPE: ['mode', 'l', 's</w>'] -> WordPiece-style: ['mode', '##l', '##s</w>']
robust -> BPE: ['r', 'o', 'b', 'u', 's', 't'] -> WordPiece-style: ['r', '##o', '##b', '##u', '##s', '##t']

=== 3) 임베딩(공간) ===
tokens: ['tokenization</w>']
ids   : [2]
vecs shape: (1, 8)

=== 4) 포지션 + 마스크(Self-Attention) ===
ids_padded: [2, 0, 0, 0, 0, 0]
mask      : [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
attention row0(sum): 0.9999999989999999
attention row0 to PAD idx: [0.0, 0.0, 0.0, 0.0, 0.0]
no